In [ ]:
# modular exponentiation: a^m = 1 mode n
# if a and n are coprime, then there is at least one integer m that satisfies that equation where m = phi(n)
# where phi(n) is the Euler's totient function

# order finding problem:
# find the smallest positive integer r such that a^r = 1 mod n
# r is called the order of a modulo n
# if a and n are coprime, then the order of a modulo n divides phi(n)
# phi(n) % r = 0

# this method succeeds in finding a factor of N with probability at least 1/2
# providing N is odd and not a prime power

# the only pair of 3-bit numbers that satisfy the order finding problem is (a=5, N=6)

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_distribution
from fractions import Fraction
from copy import deepcopy
import numpy as np

In [2]:
def carry():
    qc = QuantumCircuit(4, name="carry")
    qc.ccx(1, 2, 3)
    qc.cx(1, 2)
    qc.ccx(0, 2, 3)
    return qc.to_gate()

In [3]:
def sum():
    qc = QuantumCircuit(3, name="sum")
    qc.cx(1, 2)
    qc.cx(0, 2)
    return qc.to_gate()

In [4]:
def adder(n):

    qc = QuantumCircuit(3 * n + 1, name="adder")

    sc = 0
    for i in range(n - 1):
        qc.append(carry(), list(range(sc, sc + 4)))
        sc += 3

    qc.append(carry(), list(range(sc, sc + 4)))

    for i in range(n):
        if i == 0:
            qc.cx(sc + 1, sc + 2)
        qc.append(sum(), list(range(sc, sc + 3)))
        sc -= 3
        if sc < 0:
            break
        qc.append(carry().inverse(), list(range(sc, sc + 4)))

    return qc.to_gate()

In [5]:
def adder_mod(indices):

    # 0 <= a, b < N

    c, a, b, N1, N2, t = indices
    n = len(a)
    qubits = 6 * n + 2

    qc = QuantumCircuit(qubits, name="adder_mod")

    # adder
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(a[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n), lines)

    # subtractor
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(N1[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n).inverse(), lines)

    # check overflow
    qc.x(c[-1])
    qc.cx(c[-1], t[0])
    qc.x(c[-1])

    # reset N1 to 0 if t is 1
    for i in range(n):
        qc.ccx(t[0], N2[i], N1[i])

    # adder
    qc.append(adder(n), lines)

    # restore N1 to original value if t is 1
    for i in range(n):
        qc.ccx(t[0], N2[i], N1[i])

    # subtractor
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(a[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n).inverse(), lines)

    # restore t to 0
    qc.cx(c[-1], t[0])

    # adder
    qc.append(adder(n), lines)

    return qc.to_gate()

In [6]:
def mult_mod(indices):

    c, a, b, N1, N2, t, r, x = indices
    n = len(a)
    qubits = 6 * n + 3
    # qubits = 7 * n + 2

    qc = QuantumCircuit(qubits, name="mult_mod")

    # default order: (a + b) mod N
    default = [c, a, b, N1, N2, t, r]
    reg = []
    for i in default:
        for j in i:
            reg.append(j)

    # (a + r) mod N
    # if x[n - 1] == "1":
    # order = [c, a, r, N1, N2, t]
    # qc.append(adder_mod(order), reg)

    order = [c, a, r, N1, N2, t]
    U = adder_mod(order).control(1)
    qc.append(U, [x[0]] + reg)

    # iterations = 1
    # for i in range(1, n):
    #     for _ in range(iterations):
    #         # (a, b) mode N
    #         order = [c, a, b, N1, N2, t]
    #         U = adder_mod(order)
    #         qc.append(U, reg)
    #     iterations *= 2
    #     # (b + r) mod N
    #     if x[n - i - 1] == "1":
    #         order = [c, b, r, N1, N2, t]
    #         # qc.append(adder_mod(order), reg)
    #         U = adder_mod(order).control(1)
    #         qc.append(U, [x[i]] + reg)

    return qc.to_gate()

In [7]:
def calculate(
    a: str = "",
    b: str = "",
    N: str = "",
    x: str = "",
    operation: str = "add",
    ancilla: int = 1,
):

    if operation not in ["add", "add_mod", "mult_mod", "factor"]:
        raise ValueError("Invalid operation")

    if operation == "mult_mod" or operation == "factor":
        b = deepcopy(a)

    n = max(len(a), len(b), len(N))
    a = a.zfill(n)
    b = b.zfill(n)
    N = N.zfill(n)
    x = x.zfill(n)

    qubits = 6 * n + 3
    # qubits = 7 * n + 2

    c_qubits = list(range(n + 1))  # n + 1 qubits
    a_qubits = list(range(n + 1, 2 * n + 1, 1))  # n qubits
    b_qubits = list(range(2 * n + 1, 3 * n + 1, 1))  # n qubits
    N1_qubits = list(range(3 * n + 1, 4 * n + 1, 1))  # n qubits
    N2_qubits = list(range(4 * n + 1, 5 * n + 1, 1))  # n qubits
    t = [5 * n + 1]  # 1 qubit
    r_qubits = list(range(5 * n + 2, 6 * n + 2, 1))  # n qubits
    x_qubits = [6 * n + 2]
    # x_qubits = list(range(6 * n + 2, 7 * n + 2, 1))  # n qubits

    if operation == "factor":
        ancilla_reg = QuantumRegister(ancilla)
        qreg = QuantumRegister(qubits)
        creg = ClassicalRegister(ancilla)
        qc = QuantumCircuit(qreg, ancilla_reg, creg)
        qc.h(ancilla_reg)
    else:
        if operation == "add":
            creg = ClassicalRegister(n + 1)
        else:
            creg = ClassicalRegister(n)
        qreg = QuantumRegister(qubits)
        qc = QuantumCircuit(qreg, creg)

    # c
    ptr = n + 1

    # a1
    for i in range(n):
        if a[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # b
    for i in range(n):
        if b[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # N1
    for i in range(n):
        if N[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # N2
    for i in range(n):
        if N[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # t
    ptr += 1

    # a2
    ptr += n

    # x
    if x[n - 1] == "1":
        qc.x(ptr)
    # for i in range(n):
    #     if x[n - i - 1] == "1":
    #         qc.x(ptr)
    #     ptr += 1

    if operation == "add":
        lines = []
        for i in range(n):
            lines.append(c_qubits[i])
            lines.append(a_qubits[i])
            lines.append(b_qubits[i])
        lines.append(c_qubits[-1])
        qc.append(adder(n), lines)
        qc.measure(b_qubits + [c_qubits[-1]], creg)

    if operation == "add_mod":
        indices = [c_qubits, a_qubits, b_qubits, N1_qubits, N2_qubits, t, r_qubits]
        reg = []
        for i in indices:
            for j in i:
                reg.append(j)
        qc.append(
            adder_mod(
                [c_qubits, a_qubits, b_qubits, N1_qubits, N2_qubits, t, r_qubits]
            ),
            reg,
        )
        qc.measure(b_qubits, creg)

    if operation == "mult_mod":
        indices = [
            c_qubits,
            a_qubits,
            b_qubits,
            N1_qubits,
            N2_qubits,
            t,
            r_qubits,
            # x_qubits,
        ]
        qc.append(mult_mod(x, indices), qreg)
        qc.measure(r_qubits, creg)

    if operation == "factor":
        indices = [
            c_qubits,
            a_qubits,
            b_qubits,
            N1_qubits,
            N2_qubits,
            t,
            r_qubits,
            x_qubits,
        ]
        U = mult_mod(indices).control(1)
        for i in range(ancilla):
            for _ in range(2**i):
                qc.append(U, [ancilla_reg[i]] + [*qreg])
        qc.append(QFT(ancilla).inverse(), ancilla_reg)
        qc.measure(ancilla_reg, creg)

    return qc

In [24]:
# a = "011"
# b = "110"
# qc = calculate(a=a, b=b, operation="add")

In [25]:
# a = "011"
# b = "110"
# N = "111"
# qc = calculate(a=a, b=b, N=N, operation="add_mod")

In [26]:
# a = "1101"
# x = "1010"
# N = "1111"
# qc = calculate(a=a, x=x, N=N, operation="mult_mod")

In [27]:
# qc = transpile(qc, AerSimulator(), optimization_level=3)

In [28]:
# job = AerSimulator().run(qc, shots=1)
# counts = job.result().get_counts(qc)
# plot_distribution(counts)

### Factorization


In [8]:
def order_finding(qc, ancilla, N):
    simulator = AerSimulator()
    isa_circuit = transpile(qc, simulator)
    result = simulator.run(isa_circuit).result()
    counts = result.get_counts()
    highest_probability_outcome = max(counts, key=counts.get)
    phase = int(highest_probability_outcome, 2) / 2**ancilla  # theta = s / r
    r = Fraction(phase).limit_denominator(N).denominator
    return r

In [9]:
def factorization(ancilla, N):
    Zn = [i for i in range(2, N)]
    Zn_star = [i for i in Zn if np.gcd(i, N) == 1]
    i = 0
    while True:
        i += 1
        # a = np.random.choice(Zn)
        a = np.random.choice(Zn_star)
        print(f"ATTEMPT {i}: a = {a}")
        d = np.gcd(a, N)
        if d >= 2:
            return d, N // d
        qc = calculate(
            a=np.binary_repr(a),
            x="1",
            N=np.binary_repr(N),
            operation="factor",
            ancilla=ancilla,
        )
        r = order_finding(qc, ancilla, N)
        if r % 2 == 0:
            d = np.gcd(a ** (r // 2) - 1, N)
            if d >= 2:
                print(f"order of {a} mod {N} is {r}")
                return d, N // d
        print("FAILED")

In [41]:
N = 4
factors = factorization(ancilla=1, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 3
FAILED
ATTEMPT 2: a = 3
order of 3 mod 4 is 2
factors of 4: (2, 2)


In [40]:
N = 6
factors = factorization(ancilla=1, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 5
FAILED
ATTEMPT 2: a = 5
FAILED
ATTEMPT 3: a = 5
order of 5 mod 6 is 2
factors of 6: (2, 3)


In [42]:
N = 12
factors = factorization(ancilla=1, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 7
FAILED
ATTEMPT 2: a = 11
FAILED
ATTEMPT 3: a = 5
FAILED
ATTEMPT 4: a = 7
order of 7 mod 12 is 2
factors of 12: (6, 2)


In [43]:
N = 15
factors = factorization(ancilla=2, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 11
FAILED
ATTEMPT 2: a = 4
FAILED
ATTEMPT 3: a = 7
order of 7 mod 15 is 4
factors of 15: (3, 5)


In [35]:
N = 15
factors = factorization(ancilla=2, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 2
order of 2 mod 15 is 4
factors of 15: (3, 5)
